<a href="https://colab.research.google.com/github/HirenOrdex/ets_doc/blob/main/_swagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/HirenOrdex/ets_doc.git

Cloning into 'ets_doc'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 4), reused 7 (delta 2), pack-reused 0
Receiving objects: 100% (15/15), 515.00 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [15]:
import os
import json

def process_properties(properties, required_properties, indent_level=0):
    """
    Recursively processes the properties of a schema object to generate the correct Swagger comments.
    """
    indent = " " * indent_level
    comment = ""

    for prop_name, prop_value in properties.items():
        prop_type = prop_value.get("type", "object")
        example = prop_value.get("example", "")
        required = prop_name in required_properties
        comment += f"{indent}{prop_name}:\n"
        comment += f"{indent}  type: {prop_type}\n"
        comment += f"{indent}  required: {required}\n"
        if example:
            comment += f"{indent}  example: {example}\n"
        if prop_type == "object" and "properties" in prop_value:
            comment += f"{indent}  properties:\n"
            comment += process_properties(prop_value["properties"], prop_value.get('required', []), indent_level + 4)
        elif prop_type == "array" and "items" in prop_value:
            comment += f"{indent}  items:\n"
            comment += process_properties(prop_value["items"]["properties"], prop_value["items"].get('required', []), indent_level + 4)

    return comment

def generate_swagger_comments(swagger_file):
    with open(swagger_file, 'r', encoding='utf-8') as f:
        swagger_data = json.load(f)

    paths = swagger_data.get('paths', {})
    comments = []

    for path_key, path_value in paths.items():
        for method_key, method_value in path_value.items():
            tags = method_value.get('tags', [])
            summary = method_value.get('summary', '')
            deprecated = method_value.get('deprecated', False)
            description = method_value.get('description', '')
            parameters = method_value.get('parameters', [])
            responses = method_value.get('responses', {})
            request_body = method_value.get('requestBody', {})

            # Generate comment for each route
            comment = "/**\n"
            comment += f" * @swagger\n"
            comment += f" * paths:\n"
            comment += f" *   {path_key}:\n"
            comment += f" *     {method_key.lower()}:\n"
            comment += f" *       tags:\n"
            for tag in tags:
                comment += f" *         - {tag}\n"
            comment += f" *       summary: {summary}\n"
            if deprecated:
                comment += f" *       deprecated: {deprecated}\n"
            if description:
                comment += f" *       description: {description}\n"
            if parameters:
                comment += f" *       parameters:\n"
                for param in parameters:
                    comment += f" *         - name: {param['name']}\n"
                    comment += f" *           in: {param['in']}\n"
                    comment += f" *           required: {param.get('required', False)}\n"
                    comment += f" *           type: {param.get('type', 'string')}\n"
                    if 'x-example' in param:
                        comment += f" *           example: {param['x-example']}\n"
            if request_body:
                content = request_body.get('content', {})
                for content_type, content_details in content.items():
                    schema = content_details.get('schema', {})
                    if schema:
                        comment += f" *       requestBody:\n"
                        comment += f" *         required: true\n"
                        comment += f" *         content:\n"
                        comment += f" *           {content_type}:\n"
                        comment += f" *             schema:\n"
                        comment += f" *               type: {schema.get('type', 'object')}\n"
                        if 'properties' in schema:
                            comment += f" *               properties:\n"
                            comment += process_properties(schema["properties"], schema.get('required', []), 18)

            if responses:
                comment += f" *       responses:\n"
                for code, resp in responses.items():
                    comment += f" *         '{code}':\n"
                    comment += f" *           description: {resp.get('description', '')}\n"
            comment += f" */\n"

            comments.append(comment)

    return comments

def process_folder(folder_path):
    # Ensure folder path ends with a slash
    if not folder_path.endswith('/'):
        folder_path += '/'

    # List all files in the folder
    try:
        files = os.listdir(folder_path)
        print(f"Files in the folder '{folder_path}': {files}")
    except Exception as e:
        print(f"Error accessing folder '{folder_path}': {e}")
        return

    # Filter JSON files
    json_files = [f for f in files if f.endswith('.json')]
    if not json_files:
        print(f"No JSON files found in the folder '{folder_path}'.")
        return

    # Process each JSON file
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        print(f"Generating Swagger comments for {file_path}...")

        # Generate comments for the JSON file
        try:
            swagger_comments = generate_swagger_comments(file_path)
            # Write comments to a txt file
            txt_file_path = os.path.splitext(file_path)[0] + '_comments.txt'
            with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
                for comment in swagger_comments:
                    txt_file.write(comment)

            print(f"Comments saved to {txt_file_path}")
        except Exception as e:
            print(f"Error processing file '{file_path}': {e}")

if __name__ == "__main__":
    api_folder = "/content/ets_doc/api"  # Replace with your API folder path

    if os.path.exists(api_folder):
        process_folder(api_folder)
    else:
        print(f"API folder '{api_folder}' not found.")


Files in the folder '/content/ets_doc/api/': ['driver.json', '.ipynb_checkpoints', 'driver_comments.txt', 'trip_comments.txt', 'auth_comments.txt', 'json.json', 'trip.json', 'auth.json']
Generating Swagger comments for /content/ets_doc/api/driver.json...
Comments saved to /content/ets_doc/api/driver_comments.txt
Generating Swagger comments for /content/ets_doc/api/json.json...
Comments saved to /content/ets_doc/api/json_comments.txt
Generating Swagger comments for /content/ets_doc/api/trip.json...
Comments saved to /content/ets_doc/api/trip_comments.txt
Generating Swagger comments for /content/ets_doc/api/auth.json...
Comments saved to /content/ets_doc/api/auth_comments.txt
